# Akıllı Çöp Kutusu Veri Analizi

Bu notebook akıllı çöp kutularının doluluk verilerini analiz eder.

## 1. Kutuphaneler

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans

## 2. Veri Yukleme

In [ ]:
df = pd.read_csv("Smart_Bin.csv")
print(f"Toplam kayit: {len(df)}")
df.head()

In [ ]:
print("Eksik veriler:")
print(df.isnull().sum())
print(f"\nKonteyner turu sayisi: {df['Container Type'].nunique()}")
print(f"Atik turleri: {df['Recyclable fraction'].unique()}")

## 3. Pivot Tablo ve Korelasyon

In [ ]:
pivot = pd.pivot_table(df, values='FL_B', index='Container Type', 
                       columns='Recyclable fraction', aggfunc='mean')
pivot = pivot.round(2)
print("Doluluk Ortalamasi:")
pivot

In [ ]:
kor = df[['FL_B', 'FL_A', 'VS']].corr().round(2)
print("Korelasyon:")
kor

## 4. Grafikler

In [ ]:
# Konteyner doluluk grafigi
plt.figure(figsize=(10, 6))
ort = df.groupby('Container Type')['FL_B'].mean().sort_values()
plt.barh(ort.index, ort.values, color='green')
plt.xlabel('Doluluk Orani')
plt.title('Konteyner Turlerine Gore Doluluk')
plt.tight_layout()
plt.savefig('1_konteyner_doluluk.png')
plt.show()

In [ ]:
# Sinif karsilastirma grafigi
plt.figure(figsize=(8, 5))
sinif_ort = df.groupby('Class')['FL_B'].mean()
plt.bar(sinif_ort.index, sinif_ort.values, color=['purple', 'orange'])
plt.ylabel('Doluluk Orani')
plt.title('Sinif Bazinda Doluluk')
plt.tight_layout()
plt.savefig('3_sinif_doluluk.png')
plt.show()

## 5. Random Forest

In [ ]:
# Label Encoding
le = LabelEncoder()
df['Container_Enc'] = le.fit_transform(df['Container Type'])
df['Recyclable_Enc'] = le.fit_transform(df['Recyclable fraction'])
df['Class_Enc'] = le.fit_transform(df['Class'])

# X ve y
X = df[['FL_B', 'FL_A', 'VS', 'Container_Enc', 'Recyclable_Enc']]
y = df['Class_Enc']

# Egitim test ayirma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Egitim: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# Model egitimi
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Tahmin
tahmin = model.predict(X_test)

# Performans
print("--- RANDOM FOREST PERFORMANSI ---")
basari = accuracy_score(y_test, tahmin)
precision = precision_score(y_test, tahmin)
recall = recall_score(y_test, tahmin)
f1 = f1_score(y_test, tahmin)

print(f"Accuracy (Dogruluk): %{basari*100:.1f}")
print(f"Precision (Kesinlik): %{precision*100:.1f}")
print(f"Recall (Duyarlilik): %{recall*100:.1f}")
print(f"F1-Score: %{f1*100:.1f}")

## 6. KNN (K-En Yakin Komsu)

In [ ]:
# KNN modeli
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn_tahmin = knn.predict(X_test)

# Performans
print("--- KNN PERFORMANSI ---")
knn_basari = accuracy_score(y_test, knn_tahmin)
knn_precision = precision_score(y_test, knn_tahmin)
knn_recall = recall_score(y_test, knn_tahmin)
knn_f1 = f1_score(y_test, knn_tahmin)

print(f"Accuracy (Dogruluk): %{knn_basari*100:.1f}")
print(f"Precision (Kesinlik): %{knn_precision*100:.1f}")
print(f"Recall (Duyarlilik): %{knn_recall*100:.1f}")
print(f"F1-Score: %{knn_f1*100:.1f}")

In [ ]:
# Model Karsilastirma
print("--- MODEL KARSILASTIRMA ---")
print(f"Random Forest Accuracy: %{basari*100:.1f}")
print(f"KNN Accuracy: %{knn_basari*100:.1f}")
if basari > knn_basari:
    print("Random Forest daha iyi!")
else:
    print("KNN daha iyi!")

## 7. K-Means Kumeleme

In [ ]:
X_kume = df[['FL_B', 'FL_A', 'VS']].dropna()
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kumeler = kmeans.fit_predict(X_kume)

print("Kume Merkezleri:")
print(pd.DataFrame(kmeans.cluster_centers_, columns=['FL_B', 'FL_A', 'VS']).round(2))

print("\nKume Dagilimi:")
print(pd.Series(kumeler).value_counts().sort_index())

## 8. Sonuclar

In [ ]:
print("="*40)
print("ANALIZ TAMAMLANDI")
print("="*40)
print(f"\nVeri Seti: {len(df)} kayit")
print(f"Konteyner Turu: {df['Container Type'].nunique()} cesit")
print(f"\nRandom Forest Accuracy: %{basari*100:.1f}")
print(f"KNN Accuracy: %{knn_basari*100:.1f}")
print(f"K-Means Kume Sayisi: 3")